# Constraint Optimization

*Constraint optimization*, or *constraint programming* (CP), is the name given to identifying feasible solutions out of a very large set of candidates, where the problem can be modeled in terms of arbitrary constraints. CP problems arise in many scientific and engineering disciplines. (The word "programming" is a bit of a misnomer, similar to how "computer" once meant "a person who computes". Here, "programming" refers to the arrangement of a plan, rather than programming in a computer language.)

CP is based on feasibility (finding a feasible solution) rather than optimization (finding an optimal solution) and focuses on the constraints and variables rather than the objective function. In fact, a CP problem may not even have an objective function — the goal may simply be to narrow down a vary large set of possible solutions to a more manageable subset by adding constraints to the problem.

## 1. CP-SAT Solver

OR-Tools provides two solvers for constraint programming:

* The CP-SAT solver
* The original CP solver

The CP-SAT solver is technologically superior to the original CP solver and should be preferred in almost all situations. The exceptions are small problems for which solutions can be found quickly using either solver. In those cases you may find that the original CP solver outperforms CP-SAT.

***example: finding a feasible solution***

The following sections present several examples that illustrate how to use the CP-SAT solver. Let's start with a simple example problem in which there are:

* Three variables, x, y, and z, each of which can take on the values: 0, 1, or 2.
* One constraint: $x ≠ y$

We'll start by showing how to use the CP-SAT solver to find a single feasible solution in all four of the supported languages (Python, C++, Java, and C#). While finding a feasible solution is trivial in this case, in more complex constraint programming problems it can be very difficult to determine whether there is a feasible solution.

The CP-SAT solver returns one of the status values shown in the table below. In this example, the value returned is `OPTIMAL`:

* OPTIMAL : An optimal feasible solution was found.
* FEASIBLE : A feasible solution was found, but we don't know if it's optimal.
* INFEASIBLE: The problem was proven infeasible.
* MODEL_INVALID : The given CpModelProto didn't pass the validation step. You can get a detailed error by calling ValidateCpModel(model_proto).
* UNKNOWN : The status of the model is unknown because a search limit was reached.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from ortools.sat.python import cp_model

# creates the model.
model = cp_model.CpModel()

# creates the variables.
num_vals = 3
x = model.NewIntVar(0, num_vals - 1, 'x')
y = model.NewIntVar(0, num_vals - 1, 'y')
z = model.NewIntVar(0, num_vals - 1, 'z')

# creates the constraints.
model.Add(x != y)

# creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print('z = %i' % solver.Value(z))

x = 1
y = 0
z = 0


***example: finding an optimal solution***

Next, we'll show how to find an optimal solution to the problem in the previous section, with the additional objective: maximize $x + 2y + 3z$.

In [2]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """print intermediate solutions."""
    
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
    
    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end = ' ')
        print()
    
    def solution_count(self):
        return self.__solution_count

In [3]:
# creates the model.
model = cp_model.CpModel()

# creates the variables.
num_vals = 3
x = model.NewIntVar(0, num_vals - 1, 'x')
y = model.NewIntVar(0, num_vals - 1, 'y')
z = model.NewIntVar(0, num_vals - 1, 'z')

# creates the constraints.
model.Add(x != y)

# create a solver and solve.
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, y, z])
status = solver.SearchForAllSolutions(model, solution_printer)

print('Status = %s' % solver.StatusName(status))
print('Number of solutions found: %i' % solution_printer.solution_count())

x=0 y=1 z=0 
x=1 y=2 z=0 
x=1 y=2 z=1 
x=1 y=2 z=2 
x=1 y=0 z=2 
x=1 y=0 z=1 
x=2 y=0 z=1 
x=2 y=1 z=1 
x=2 y=1 z=2 
x=2 y=0 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=2 z=1 
x=0 y=2 z=2 
x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=1 z=0 
x=0 y=2 z=0 
Status = FEASIBLE
Number of solutions found: 18


***example: display intermediate solutions***

This section shows how to display all the intermediate solutions found by the solver while it searches for an optimal solution.

As in the section Finding an optimal solution, we create an objective function by adding the following line of code, after the constraint.

In [4]:
class VarArrayAndObjectiveSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        print('Solution %i' % self.__solution_count)
        print('  objective value = %i' % self.ObjectiveValue())
        for v in self.__variables:
            print('  %s = %i' % (v, self.Value(v)), end=' ')
        print()
        self.__solution_count += 1

    def solution_count(self):
        return self.__solution_count

In [5]:
# creates the model.
model = cp_model.CpModel()

# creates the variables.
num_vals = 3
x = model.NewIntVar(0, num_vals - 1, 'x')
y = model.NewIntVar(0, num_vals - 1, 'y')
z = model.NewIntVar(0, num_vals - 1, 'z')

# creates the constraints.
model.Add(x != y)

model.Maximize(x + 2 * y + 3 * z)

# creates a solver and solves.
solver = cp_model.CpSolver()
solution_printer = VarArrayAndObjectiveSolutionPrinter([x, y, z])
status = solver.SolveWithSolutionCallback(model, solution_printer)

print('Status = %s' % solver.StatusName(status))
print('Number of solutions found: %i' % solution_printer.solution_count())

Solution 0
  objective value = 2
  x = 0   y = 1   z = 0 
Solution 1
  objective value = 4
  x = 0   y = 2   z = 0 
Solution 2
  objective value = 7
  x = 0   y = 2   z = 1 
Solution 3
  objective value = 10
  x = 0   y = 2   z = 2 
Solution 4
  objective value = 11
  x = 1   y = 2   z = 2 
Status = OPTIMAL
Number of solutions found: 5


## 2. Original CP Solver

(CP-SAT solver is recommended rather than the original cp solver)

The decision builder is the main input to the original CP solver. It contains the following:

* vars — An array containing the variables for the problem.
* A rule for choosing the next variable to assign a value to.
* A rule for choosing the next value to assign to that variable.

In [6]:
from __future__ import print_function
import sys
from ortools.constraint_solver import pywrapcp

def print_solution(solver, x, y, z):
    count = 0
    
    while solver.NextSolution():
        count += 1
        print("x =", x.Value(), "y =", y.Value(), "z =", z.Value())
    print("\nNumber of solutions found:", count)

solver = pywrapcp.Solver("simple_example")

# create the variables
num_vals = 3
x = solver.IntVar(0, num_vals - 1, "x")
y = solver.IntVar(0, num_vals - 1, "y")
z = solver.IntVar(0, num_vals - 1, "z")

# create the constraints.
solver.Add(x != y)

# call the solver.
db = solver.Phase([x, y, z], solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)
solver.Solve(db)
print_solution(solver, x, y, z)

x = 0 y = 1 z = 0
x = 0 y = 1 z = 1
x = 0 y = 1 z = 2
x = 0 y = 2 z = 0
x = 0 y = 2 z = 1
x = 0 y = 2 z = 2
x = 1 y = 0 z = 0
x = 1 y = 0 z = 1
x = 1 y = 0 z = 2
x = 1 y = 2 z = 0
x = 1 y = 2 z = 1
x = 1 y = 2 z = 2
x = 2 y = 0 z = 0
x = 2 y = 0 z = 1
x = 2 y = 0 z = 2
x = 2 y = 1 z = 0
x = 2 y = 1 z = 1
x = 2 y = 1 z = 2

Number of solutions found: 18


The main input to the original CP solver is the decision builder, which contains the variables for the problem and sets options for the solver.

The code example above creates a decision builder using the Phase method (corresponding to the C++ method MakePhase).

The term "Phase" refers to a phase of the search. In this simple example, there is just one phase, but for more complex problems, the decision builder can have more than one phase, so that the solver can employ different search strategies from one phase to the next.

The Phase method has three input parameters:

* vars — An array containing the variables for the problem, which in this case is [x, y, z].
* IntVarStrategy — The rule for choosing the next unbound variable to assign a value. Here, the code uses the default CHOOSE_FIRST_UNBOUND, which means that at each step, the solver selects the first unbound variable in the order they occur in the variable array passed to the Phase method.
* IntValueStrategy — The rule for choosing the next value to assign to a variable. Here the code uses the default ASSIGN_MIN_VALUE, which selects the smallest value that hasn't already been tried for the variable. This assigns values in increasing order. Another option is ASSIGN_MAX_VALUE, in which case the solver would assign values in decreasing order.

## 3. Cryptarithmetic Puzzles 

A cryptarithmetic puzzle is a mathematical exercise where the digits of some numbers are represented by letters (or symbols). Each letter represents a unique digit. The goal is to find the digits such that a given mathematical equation is verified:

CP + IS + FUN =   TRUE

One assignment of letters to digits yields the following equation:

23 + 74 + 968 = 1065

As with any optimization problem, we'll start by identifying variables and constraints. The variables are the letters, which can take on any single digit value.

For CP + IS + FUN = TRUE, the constraints are as follows:

* The equation: CP + IS + FUN = TRUE.
* Each of the ten letters must be a different digit.
* C, I, F, and T can't be zero (since we don't write leading zeros in numbers).

***CP-SAT***

In [7]:
from __future__ import print_function

from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [8]:
# constraint programming engine
model = cp_model.CpModel()

base =  10

c = model.NewIntVar(1, base - 1, 'C')
p = model.NewIntVar(0, base - 1, 'P')
i = model.NewIntVar(1, base - 1, 'I')
s = model.NewIntVar(0, base - 1, 'S')
f = model.NewIntVar(1, base - 1, 'F')
u = model.NewIntVar(0, base - 1, 'U')
n = model.NewIntVar(0, base - 1, 'N')
t = model.NewIntVar(1, base - 1, 'T')
r = model.NewIntVar(0, base - 1, 'R')
e = model.NewIntVar(0, base - 1, 'E')

# we need to group variables in a list to use the constraint AllDifferent.
letters = [c, p, i, s, f, u, n, t, r, e]

# verify that we have enough digits.
assert base >= len(letters)

# define constraints.
model.AddAllDifferent(letters)

# CP + IS + FUN = TRUE
model.Add(c * base + p + i * base + s + f * base * base + u * base + n ==
              t * base * base * base + r * base * base + u * base + e)

# solve model.
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(letters)
status = solver.SearchForAllSolutions(model, solution_printer)

print()
print('Statistics')
print('  - status          : %s' % solver.StatusName(status))
print('  - conflicts       : %i' % solver.NumConflicts())
print('  - branches        : %i' % solver.NumBranches())
print('  - wall time       : %f s' % solver.WallTime())
print('  - solutions found : %i' % solution_printer.solution_count())

C=2 P=3 I=7 S=4 F=9 U=6 N=8 T=1 R=0 E=5 
C=2 P=4 I=7 S=3 F=9 U=6 N=8 T=1 R=0 E=5 
C=2 P=5 I=7 S=3 F=9 U=4 N=8 T=1 R=0 E=6 
C=2 P=8 I=7 S=3 F=9 U=4 N=5 T=1 R=0 E=6 
C=2 P=8 I=7 S=3 F=9 U=6 N=4 T=1 R=0 E=5 
C=3 P=7 I=6 S=2 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=7 I=3 S=2 F=9 U=8 N=5 T=1 R=0 E=4 
C=6 P=5 I=3 S=2 F=9 U=8 N=7 T=1 R=0 E=4 
C=3 P=5 I=6 S=2 F=9 U=8 N=7 T=1 R=0 E=4 
C=3 P=8 I=6 S=4 F=9 U=2 N=5 T=1 R=0 E=7 
C=3 P=7 I=6 S=5 F=9 U=8 N=2 T=1 R=0 E=4 
C=3 P=8 I=6 S=5 F=9 U=2 N=4 T=1 R=0 E=7 
C=3 P=5 I=6 S=4 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=4 I=6 S=5 F=9 U=2 N=8 T=1 R=0 E=7 
C=3 P=2 I=6 S=5 F=9 U=8 N=7 T=1 R=0 E=4 
C=3 P=4 I=6 S=8 F=9 U=2 N=5 T=1 R=0 E=7 
C=3 P=2 I=6 S=7 F=9 U=8 N=5 T=1 R=0 E=4 
C=3 P=5 I=6 S=8 F=9 U=2 N=4 T=1 R=0 E=7 
C=3 P=5 I=6 S=7 F=9 U=8 N=2 T=1 R=0 E=4 
C=2 P=5 I=7 S=6 F=9 U=8 N=3 T=1 R=0 E=4 
C=2 P=5 I=7 S=8 F=9 U=4 N=3 T=1 R=0 E=6 
C=2 P=6 I=7 S=5 F=9 U=8 N=3 T=1 R=0 E=4 
C=2 P=4 I=7 S=8 F=9 U=6 N=3 T=1 R=0 E=5 
C=2 P=3 I=7 S=8 F=9 U=6 N=4 T=1 R=0 E=5 
C=2 P=8 I=7 S=5 

***original CP***

Next, we'll present the solution using the original CP solver.

In this case we'll treat the base as a variable, so you can solve the equation for higher bases. (There can be no lower base solutions for CP + IS + FUN = TRUE since the ten letters must all be different.)

In [12]:
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from os import abort

# Constraint programming engine
solver = pywrapcp.Solver('CP is fun!');

kBase = 10

# Decision variables.
digits = list(range(0, kBase))
digits_without_zero = list(range(1, kBase))
c = solver.IntVar(digits_without_zero, 'C');
p = solver.IntVar(digits, 'P');
i = solver.IntVar(digits_without_zero, 'I');
s = solver.IntVar(digits, 'S');
f = solver.IntVar(digits_without_zero, 'F');
u = solver.IntVar(digits, 'U');
n = solver.IntVar(digits, 'N');
t = solver.IntVar(digits_without_zero, 'T');
r = solver.IntVar(digits, 'R');
e = solver.IntVar(digits, 'E');

# We need to group variables in a list to use the constraint AllDifferent.
letters = [c, p, i, s, f, u, n, t, r, e]

# Verify that we have enough digits.
assert kBase >= len(letters)

# Define constraints.
solver.Add(solver.AllDifferent(letters))

# CP + IS + FUN = TRUE
solver.Add (p + s + n + kBase * (c + i + u) + kBase * kBase * f ==
              e + kBase * u + kBase * kBase * r + kBase * kBase * kBase * t)

db = solver.Phase(letters, solver.INT_VAR_DEFAULT,
                             solver.INT_VALUE_DEFAULT)
solver.NewSearch(db)

while solver.NextSolution():
    print(letters)
    # Is CP + IS + FUN = TRUE?
    assert (kBase*c.Value() +  p.Value() + kBase*i.Value() + s.Value() +
            kBase*kBase*f.Value() + kBase*u.Value() + n.Value() ==
            kBase*kBase*kBase*t.Value() + kBase*kBase*r.Value() +
            kBase*u.Value() + e.Value())

solver.EndSearch()

[C(2), P(3), I(7), S(4), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(3), I(7), S(5), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(5), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(6), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(8), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(8), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(3), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(8), F(9), U(6), N(3), T(1), R(0), E(5)]
[C(2), P(5), I(7), S(3), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(5), I(7), S(3), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(6), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(8), F(9), U(4), N(3), T(1), R(0), E(6)]
[C(2), P(6), I(7), S(3), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(6), I(7), S(5), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(8), I(7), S(3), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(8), I(7), S(3), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(8), I(7), S(4),